In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from matplotlib import pyplot

In [2]:
fnames = ["D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/amazon1_Flow.csv",
          "D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/amazon2_Flow.csv",
          "D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/amazon3_Flow.csv",
          "D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/amazon4_Flow.csv",
          "D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/amazon5_Flow.csv",
          "D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/YouTube_1_Flow.csv",
          "D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/YouTube_2_Flow.csv",
          "D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/YouTube_3_Flow.csv",
          "D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/YouTube_4_Flow.csv",
          "D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/YouTube_5_Flow.csv",
          "D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/YouTube_6_Flow.csv",
          "D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/YouTube_7_Flow.csv",
          "D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/YouTube_8_Flow.csv",
          "D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/YouTube_9_Flow.csv",
          
         ]

In [3]:
dfList = list()
for fname in fnames: 
    data = pd.read_csv(fname)
    print('file:', fname, ' data size: ',data.shape)
    dfList.append(data)
############## combine the data frames
masterDF = pd.concat(dfList)
print(masterDF.shape)

file: D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/amazon1_Flow.csv  data size:  (870, 84)
file: D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/amazon2_Flow.csv  data size:  (516, 84)
file: D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/amazon3_Flow.csv  data size:  (921, 84)
file: D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/amazon4_Flow.csv  data size:  (467, 84)
file: D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/amazon5_Flow.csv  data size:  (800, 84)
file: D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/YouTube_1_Flow.csv  data size:  (1420, 84)
file: D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/filteredPcapCSV/VideoSharing_filteredpcapcsv/YouTube_2_Flow.csv  data size:  (52, 84)
file: D:/New pcaps_IoTSpyCamer

In [4]:
#see unique labels
masterDF['Label'].unique()
masterDF['Label'].replace('VConf_Gmeet','VConf_GMeet',inplace=True)
uniqueLabels = list(masterDF['Label'].unique())
print(uniqueLabels)

['VShare_Amazon', 'VShare_YouTube']


In [5]:
#discard all those columns that helps in uniquely identifying a flow
masterPruned = masterDF.iloc[:,np.arange(7,84)]
print('pruneddata size: ', masterPruned.shape)
ftList = list(masterPruned.columns)
print(ftList)

pruneddata size:  (7880, 77)
['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Fwd Byts/b Avg', 'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg', 'Bwd Byts/b Avg

In [6]:
#discard those flows that has 0 flow duration
masterSelected = masterPruned.drop(masterPruned[masterPruned['Flow Duration']==0].index)
masterSelected.shape

(5285, 77)

In [7]:
#identify those columns that has non-numeric values
typesList = masterSelected.dtypes
ftCount = len(ftList)
nonNumericFtList = list()
for fi in range(ftCount):
    if ftList[fi] != 'Label' and typesList[fi] != 'int64' and typesList[fi] != 'float64':
        print(ftList[fi], '  ', typesList[fi])
        nonNumericFtList.append(ftList[fi])
print(nonNumericFtList)

[]


In [8]:
#pyplot.boxplot(results, labels=names, showmeans=True)
#pyplot.scatter(fpackets,fbytes)
#pyplot.show()


In [9]:
for ft in nonNumericFtList:
    masterSelected[ft] = pd.to_numeric(masterSelected[ft], errors='coerce')

In [10]:
print(nonNumericFtList)
for ft in nonNumericFtList:
    countNaN = masterSelected[ft].isna().sum()
    print('converted to numeric ft: ', ft, '  countNaN: ', countNaN)

[]


In [11]:
number_inf = masterSelected[masterSelected == np.inf].count().sum()
print('count inf: ',number_inf)

count inf:  0


In [12]:
typesList = masterSelected.dtypes
ftCount = len(ftList)
nonNumericFtList = list()
for fi in range(ftCount):
    if ftList[fi] != 'Label' and typesList[fi] != 'int64' and typesList[fi] != 'float64':
        print(ftList[fi], '  ', typesList[fi])
        nonNumericFtList.append(ftList[fi])
print(nonNumericFtList)

[]


In [13]:
#identify those columns that has 0 std
count = 0
staticFtList = list()
for ft in ftList:
    if ft == 'Label':
        continue
    #print('ft : ', ft)
    ftMin = masterSelected[ft].min()
    ftMax = masterSelected[ft].max()
    #std = masterPruned[ft].std()
    
    if ftMin == ftMax:
        count += 1
        print('count: ', count, ' ft: ', ft, ' ftmin: ', ftMin, ' ftMax: ', ftMax)
        staticFtList.append(ft)
print(staticFtList)

count:  1  ft:  Fwd PSH Flags  ftmin:  0  ftMax:  0
count:  2  ft:  Fwd URG Flags  ftmin:  0  ftMax:  0
count:  3  ft:  Bwd URG Flags  ftmin:  0  ftMax:  0
count:  4  ft:  URG Flag Cnt  ftmin:  0  ftMax:  0
count:  5  ft:  CWE Flag Count  ftmin:  0  ftMax:  0
count:  6  ft:  ECE Flag Cnt  ftmin:  0  ftMax:  0
count:  7  ft:  Fwd Byts/b Avg  ftmin:  0  ftMax:  0
count:  8  ft:  Fwd Pkts/b Avg  ftmin:  0  ftMax:  0
count:  9  ft:  Fwd Blk Rate Avg  ftmin:  0  ftMax:  0
count:  10  ft:  Bwd Byts/b Avg  ftmin:  0  ftMax:  0
count:  11  ft:  Bwd Pkts/b Avg  ftmin:  0  ftMax:  0
count:  12  ft:  Bwd Blk Rate Avg  ftmin:  0  ftMax:  0
count:  13  ft:  Init Fwd Win Byts  ftmin:  -1  ftMax:  -1
count:  14  ft:  Fwd Seg Size Min  ftmin:  0  ftMax:  0
['Fwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Fwd Byts/b Avg', 'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg', 'Bwd Byts/b Avg', 'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg', 'Init Fwd Win Byts', 'Fwd Seg Size 

In [14]:
#remove those columns that have 0 std
print('before droping column: ', masterSelected.shape)
for ft in staticFtList:
    masterSelected.drop([ft], axis=1,inplace=True)
print('after droping column: ', masterSelected.shape)    


before droping column:  (5285, 77)
after droping column:  (5285, 63)


In [15]:
xCount = masterSelected.shape[1]
print(xCount)
finalFtList = list(masterSelected.columns)
print(finalFtList)

63
['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Bwd PSH Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Subflow Fwd Pkts', 'Subflow Fwd Byts', 'Subflow Bwd Pkts', 'Subflow Bwd Byts', 'Init Bwd Win Byts', 'Fwd Act Data Pkts', 'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean', 'Idle 

In [16]:
#report intermediate data frame to a csv
masterSelected.to_csv('D:/New pcaps_IoTSpyCamera/TDSC_WorkAfterReview/lineplotCSV_Vshare/ProcessedNPrunnedAllShare_20122022.csv', index = False)

In [17]:
#classification with all the features
masterDS = masterSelected.values
X = masterDS[:,:-1]
Y = masterDS[:,-1]
print("Xshape: ", X.shape)
print("Yshape: ", Y.shape)

Xshape:  (5285, 62)
Yshape:  (5285,)


In [18]:
#import time

#start_time = time.time()
#print(f'{start_time:.2f}s start time')

In [19]:
clf = DecisionTreeClassifier(random_state=10)
start_time = time.time()
print(f'{start_time:.2f}s start time')

#print('classification with all features:')
scores = cross_val_score(clf, X, Y, cv=10)
#print(scores)
clf.fit(X_train, Y_train)
elapsed_time = time.time() - start_time
print(f'{elapsed_time:.2f}s elapsed during training')
#print('mean score: ', scores.mean(), 'min score: ', scores.min(), 'max score: ', scores.max(), ' std score: ', scores.std())

NameError: name 'time' is not defined

In [30]:
import time
#performance with all features using decision tree
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=1)
model = DecisionTreeClassifier(random_state=10)
clf = model.fit(X_train,Y_train)
start_time = time.time()
print(f'{start_time:.2f}s start time')
yhat = model.predict(X_test)
elapsed_time = time.time() - start_time
print(f'{elapsed_time:.2f}s elapsed during training')

1673498983.49s start time
0.01s elapsed during training


In [30]:
import time

#performance with all features using decision tree
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=1)
start_time = time.time()
print(f'{start_time:.2f}s start time')
model = DecisionTreeClassifier(random_state=10)
clf = model.fit(X_train,Y_train)
elapsed_time = time.time() - start_time
print(f'{elapsed_time:.2f}s elapsed during training')
yhat = model.predict(X_test)

1673452826.44s start time
0.12s elapsed during training


In [81]:
import time

start_time = time.time()
print(f'{start_time:.2f}s start time')

1672993020.24s start time


In [107]:
#performance with all features using decision tree
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=1)
model = DecisionTreeClassifier(random_state=0)
clf = model.fit(X_train,Y_train)
yhat = model.predict(X_test)
print(yhat.shape)
print(X_train.shape)

(1745,)
(3540, 62)


In [83]:
import time

clf.fit(X_train, Y_train)
elapsed_time = time.time() - start_time
print(f'{elapsed_time:.2f}s elapsed time')

0.78s elapsed during training


In [63]:
from sklearn.tree import export_text
from sklearn import tree
#r = export_text(decision_tree, feature_names=finalFtList)
#print(r)
text_representation = tree.export_text(clf)
print(text_representation)

|--- feature_44 <= 2.50
|   |--- feature_52 <= 504.50
|   |   |--- feature_18 <= 0.50
|   |   |   |--- feature_31 <= 30.00
|   |   |   |   |--- feature_37 <= 703.34
|   |   |   |   |   |--- class: VShare_YouTube
|   |   |   |   |--- feature_37 >  703.34
|   |   |   |   |   |--- class: VShare_Amazon
|   |   |   |--- feature_31 >  30.00
|   |   |   |   |--- feature_0 <= 60893274.00
|   |   |   |   |   |--- feature_52 <= 261.50
|   |   |   |   |   |   |--- feature_18 <= -31.00
|   |   |   |   |   |   |   |--- class: VShare_YouTube
|   |   |   |   |   |   |--- feature_18 >  -31.00
|   |   |   |   |   |   |   |--- feature_54 <= 847640.50
|   |   |   |   |   |   |   |   |--- feature_29 <= 0.50
|   |   |   |   |   |   |   |   |   |--- feature_32 <= 21567.72
|   |   |   |   |   |   |   |   |   |   |--- feature_21 <= 21300000.00
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 6
|   |   |   |   |   |   |   |   |   |   |--- feature_21 >  21300000.00
|   |   |   |   |   

In [21]:
print(clf.tree_.max_depth)

21


In [22]:
share_mat = metrics.confusion_matrix(Y_test, yhat,labels=uniqueLabels)
#[i][j] ==> knwon to be in group i and predicted as group j
print(share_mat)
print('Amazon as Amazon = ',share_mat[0][0],' percent = ', (share_mat[0][0]/(share_mat[0][0]+share_mat[0][1])))
print('Amazon as Youtube = ',share_mat[0][1],' percent = ', (share_mat[0][1]/(share_mat[0][0]+share_mat[0][1])))
print('\n')

print('Youtube as Amazon = ',share_mat[1][0],' percent = ', (share_mat[1][0]/(share_mat[1][0]+share_mat[1][1])))
print('Youtube as Youtube = ',share_mat[1][1],' percent = ', (share_mat[1][1]/(share_mat[1][0]+share_mat[1][1])))


[[646  97]
 [ 93 909]]
Amazon as Amazon =  646  percent =  0.8694481830417228
Amazon as Youtube =  97  percent =  0.13055181695827725


Youtube as Amazon =  93  percent =  0.09281437125748503
Youtube as Youtube =  909  percent =  0.907185628742515


In [23]:
# feature selection using ANNOVA F-score
topKval = 10
FS = SelectKBest(score_func=f_classif, k=topKval)
#print(type(FS))
# learn relationship from training data
FS.fit(X, Y)
# transform train input data
XNew = FS.transform(X)
# transform test input data
print('XNew.shape', XNew.shape)
#print('XNew', XNew)

XNew.shape (5285, 10)


In [24]:
#find selected features
namesFilter = FS.get_support()
features =  np.array(finalFtList)
names = list()#features[0:-1]
scores = list()
for fi in range(xCount-1):
    if (fi < xCount-1) and (FS.get_support()[fi]) and (not np.isnan(FS.scores_[fi])):
        #print('i = ', fi, ' ftName: ', ftList[fi], ' FS.support_: ', FS.get_support()[fi], ' FS.scores: = ', FS.scores_[fi])
        scores.append(FS.scores_[fi])
        names.append(features[fi])
names_scores = list(zip(names, scores))
ns_df = pd.DataFrame(data = names_scores, columns=['Feat_names', 'F_Scores'])
#print(ns_df)
#Sort the dataframe for better visualization
ns_df_sorted = ns_df.sort_values(['F_Scores', 'Feat_names'], ascending = [False, True])
print('features df shape: ', ns_df_sorted.shape, '  \nfeatures df top 10 rows: \n', ns_df_sorted.head(topKval))
#restFtList = list()
#for ft in finalFtList:
#    if ft not in names:
#        restFtList.append(ft)
#    else:
#        print(ft)

features df shape:  (10, 2)   
features df top 10 rows: 
          Feat_names    F_Scores
7     Down/Up Ratio  233.918618
3   Bwd Pkt Len Min  211.518959
0     Flow Duration  161.952379
4       Fwd IAT Tot  161.350729
8  Subflow Bwd Pkts  134.081953
1      Tot Bwd Pkts  134.081953
6    Bwd Header Len  134.025846
9  Subflow Bwd Byts  106.189725
2   TotLen Bwd Pkts  106.147877
5       Bwd IAT Tot  105.064031


In [25]:
print('classification with top 10 selected features:')
scores = cross_val_score(clf, XNew, Y, cv=10)
print(scores)
print('mean score: ', scores.mean(), 'min score: ', scores.min(), 'max score: ', scores.max(), ' std score: ', scores.std())

classification with top 10 selected features:
[0.88846881 0.91304348 0.92438563 0.83364839 0.74480151 0.81060606
 0.79924242 0.78409091 0.69318182 0.77272727]
mean score:  0.8164196310935441 min score:  0.6931818181818182 max score:  0.9243856332703214  std score:  0.07076756016294754


In [28]:
#performance with to k  features using decision tree
X_train, X_test, Y_train, Y_test = train_test_split(XNew, Y, test_size=0.33, random_state=1)
model = DecisionTreeClassifier(random_state=0)
clf1 = model.fit(X_train,Y_train)
ypred = model.predict(X_test)

In [29]:
from sklearn.tree import export_text
from sklearn import tree
#r = export_text(decision_tree, feature_names=finalFtList)
#print(r)
text_representation = tree.export_text(clf1)
print(text_representation)

|--- feature_7 <= 2.50
|   |--- feature_5 <= 23.50
|   |   |--- feature_7 <= 0.50
|   |   |   |--- feature_0 <= 2604955.50
|   |   |   |   |--- feature_6 <= 26.00
|   |   |   |   |   |--- feature_2 <= 19.50
|   |   |   |   |   |   |--- feature_0 <= 214.00
|   |   |   |   |   |   |   |--- feature_0 <= 106.50
|   |   |   |   |   |   |   |   |--- feature_0 <= 100.50
|   |   |   |   |   |   |   |   |   |--- class: VShare_YouTube
|   |   |   |   |   |   |   |   |--- feature_0 >  100.50
|   |   |   |   |   |   |   |   |   |--- class: VShare_Amazon
|   |   |   |   |   |   |   |--- feature_0 >  106.50
|   |   |   |   |   |   |   |   |--- class: VShare_YouTube
|   |   |   |   |   |   |--- feature_0 >  214.00
|   |   |   |   |   |   |   |--- feature_0 <= 215.50
|   |   |   |   |   |   |   |   |--- class: VShare_Amazon
|   |   |   |   |   |   |   |--- feature_0 >  215.50
|   |   |   |   |   |   |   |   |--- feature_0 <= 2819.50
|   |   |   |   |   |   |   |   |   |--- feature_0 <= 1315.50
|   |  

In [32]:
print(clf1.tree_.max_depth)

26


In [33]:
share_mat = metrics.confusion_matrix(Y_test, ypred,labels=uniqueLabels)
#[i][j] ==> knwon to be in group i and predicted as group j

print(share_mat)
print('Amazon as Amazon = ',share_mat[0][0],' percent = ', (share_mat[0][0]/(share_mat[0][0]+share_mat[0][1])))
print('Amazon as Youtube = ',share_mat[0][1],' percent = ', (share_mat[0][1]/(share_mat[0][0]+share_mat[0][1])))
print('\n')

print('Youtube as Amazon = ',share_mat[1][0],' percent = ', (share_mat[1][0]/(share_mat[1][0]+share_mat[1][1])))
print('Youtube as Youtube = ',share_mat[1][1],' percent = ', (share_mat[1][1]/(share_mat[1][0]+share_mat[1][1])))

[[627 116]
 [152 850]]
Amazon as Amazon =  627  percent =  0.8438761776581427
Amazon as Youtube =  116  percent =  0.15612382234185734


Youtube as Amazon =  152  percent =  0.15169660678642716
Youtube as Youtube =  850  percent =  0.8483033932135728


In [98]:
#classification with unselected and uncorrelated features
corr = masterSelected.corr()
corr.shape

(62, 62)

In [99]:
corrThreshold = 0.65
unCorrUnSelFtList = list()
for ft1 in finalFtList:#baseft = 'Idle Max' for each of the selected features
    #print('ft1: ', ft1)
    isCorr = False
    for ft2 in names:# for each of the top 10 features
        if ft1 == 'Label' or ft2 == 'Label' or (ft1 == ft2):
            continue
        elif abs(corr[ft1][ft2]) > corrThreshold:
                isCorr = True
                print('ft1: ', ft1, 'ft2: ', ft2, '  corr: ', corr[ft1][ft2])
                break
    if not isCorr and ft1 not in unCorrUnSelFtList:
        #print('corr[{0}][{1}] = {2}: isPresentInRestFtlist: {3}'.format(baseft,ft,corr[baseft][ft], ft in restFtList))
        unCorrUnSelFtList.append(ft1)
        
    #elif abs(corr[ft1][ft] >= 0.6 and ft in unCorrUnSelFtList):
    #            unCorrUnSelFtList.remove(ft)
print(len(unCorrUnSelFtList))
print(unCorrUnSelFtList)

print(len(names))
print(names)

ft1:  Flow Duration ft2:  Fwd IAT Tot   corr:  0.9363974518404051
ft1:  Tot Bwd Pkts ft2:  TotLen Bwd Pkts   corr:  0.9865922453572769
ft1:  TotLen Bwd Pkts ft2:  Tot Bwd Pkts   corr:  0.9865922453572769
ft1:  Flow IAT Std ft2:  Flow Duration   corr:  0.6785223885123135
ft1:  Flow IAT Max ft2:  Flow Duration   corr:  0.7895824858160128
ft1:  Fwd IAT Tot ft2:  Flow Duration   corr:  0.9363974518404051
ft1:  Fwd IAT Std ft2:  Fwd IAT Tot   corr:  0.670739938730419
ft1:  Fwd IAT Max ft2:  Flow Duration   corr:  0.775965750693583
ft1:  Bwd IAT Tot ft2:  Flow Duration   corr:  0.9282755995528241
ft1:  Bwd IAT Std ft2:  Flow Duration   corr:  0.6652218093626003
ft1:  Bwd IAT Max ft2:  Flow Duration   corr:  0.7547757306955687
ft1:  Fwd Header Len ft2:  Tot Bwd Pkts   corr:  0.8140868674419046
ft1:  Bwd Header Len ft2:  Tot Bwd Pkts   corr:  0.999999739152537
ft1:  Subflow Bwd Pkts ft2:  Tot Bwd Pkts   corr:  1.0
ft1:  Subflow Bwd Byts ft2:  Tot Bwd Pkts   corr:  0.9865755036822564
ft1:  Idle

In [100]:
for ft1 in names:
    for ft2 in unCorrUnSelFtList:
        if ft1 != 'Label' or ft2 == 'Label':
            continue
        if abs(corr[ft][ft2]) >= corrThreshold:
            print('ft: ',ft, ' ft2: ',ft2, corr[ft][ft2])

In [101]:
#slice master data frame to only uncorrelated and unselected features
masterUnSelected = masterSelected[unCorrUnSelFtList]
print(masterUnSelected.shape)

(5285, 45)


In [102]:
#### classification with unselected columns
masterUnSelDS = masterUnSelected.values
Xnotsel = masterUnSelDS[:,:-1]
Y = masterUnSelDS[:,-1]
print("Xshape: ", Xnotsel.shape)
print("Yshape: ", Y.shape)

Xshape:  (5285, 44)
Yshape:  (5285,)


In [103]:
#find cross validation score with uncorrelated and unselecte feature data frame
clf = DecisionTreeClassifier(random_state=0)
print('classification with rest (not top 10) features:')
scores = cross_val_score(clf, X, Y, cv=10)
print(scores)
print('mean score: ', scores.mean(), ' std score: ', scores.std())

classification with rest (not top 10) features:
[0.9489603  0.94517958 0.97731569 0.91304348 0.90548204 0.73863636
 0.84848485 0.85227273 0.76704545 0.85416667]
mean score:  0.8750587157014378  std score:  0.0742583749993214


In [104]:
#classification with with uncorrelated and unselecte feature data frame
X_train, X_test, Y_train, Y_test = train_test_split(Xnotsel, Y, test_size=0.33, random_state=1)
model = DecisionTreeClassifier(random_state=0)
model.fit(X_train,Y_train)
yhat = model.predict(X_test)

In [105]:
share_mat = metrics.confusion_matrix(Y_test, yhat,labels=uniqueLabels)
#[i][j] ==> knwon to be in group i and predicted as group j

print(share_mat)
print('Amazon as Amazon = ',share_mat[0][0],' percent = ', (share_mat[0][0]/(share_mat[0][0]+share_mat[0][1])))
print('Amazon as Youtube = ',share_mat[0][1],' percent = ', (share_mat[0][1]/(share_mat[0][0]+share_mat[0][1])))
print('\n')

print('Youtube as Amazon = ',share_mat[1][0],' percent = ', (share_mat[1][0]/(share_mat[1][0]+share_mat[1][1])))
print('Youtube as Youtube = ',share_mat[1][1],' percent = ', (share_mat[1][1]/(share_mat[1][0]+share_mat[1][1])))

[[652  91]
 [ 89 913]]
Amazon as Amazon =  652  percent =  0.8775235531628532
Amazon as Youtube =  91  percent =  0.1224764468371467


Youtube as Amazon =  89  percent =  0.08882235528942116
Youtube as Youtube =  913  percent =  0.9111776447105788
